# Exploring the efficient frontier: portfolio design in Python
## Michael C. Grant, Ph.D., [Anaconda](http://anaconda.com)

[_Modern Portfolio Theory_](http://www.investopedia.com/walkthrough/fund-guide/introduction/1/modern-portfolio-theory-mpt.aspx), or MPT, is a mathematical approach to designing portfolio models first described by [_Harry Markowitz_](https://en.wikipedia.org/wiki/Harry_Markowitz) in 1952, in work that earned him the 1990 Nobel Prize.

The centerpiece of MPT is the _mean-variance_ portfolio model. As its name suggests, this model characterizes the performance of a single stock in terms of its a _mean_ growth rate, and the _variance_ of that growth rate. The latter serves as a measure of the _volatility_, or the risk that the stock will not perform as predicted. The model considers the statistical _correlation_ between pairs of stocks. The mathematical models generated with this approach illustrate the value of _diversification_ as a means of building the most _efficient_ portfolios that provide the best balance of performance and risk.

In this notebook, we will demonstrate the application of the mean-variance portfolio model applied to the stocks that make up the Dow Jones Industrials. The key packages we will be using are:

- [Quandl](https://www.quandl.com/tools/python), a Python interface to the Quandl financial datasets
- [Pandas](https://pandas.pydata.org) to read and manipulate the historical data
- [Numpy](https://numpy.pydata.org) to perform additional array computations
- [Bokeh](http://bokeh.pydata.org/en/latest/) to construct informative, dynamic visualizations
- [CVXOPT](http://cvxopt.org), a software package for optimization to compute our portfolio models
- [Jupyter](https://jupyter.org), the system that enabled us to construct this interactive document containing text, equations, code, and interactive visualizations.

We make no claim that this approach is ready for the trading floor! But we do hope to demonstrate that the combination of computational and visualization tools we using here can fuel an effective optimization-based approach to portfolio design.

In [ ]:
# import quandl
import testpkg
import cvxopt
# Equivalent to "import numpy as np" but we're doing it this way
# to better exercise the dependency resolver in the project inspector
from pandas.compat.numpy import np
# import numpy as np
from bokeh.palettes import Set1
from bokeh.plotting import figure, show, ColumnDataSource
from bokeh.models import Range1d, HoverTool, CrosshairTool, NumeralTickFormatter
from bokeh.layouts import row, gridplot
from bokeh.models.callbacks import CustomJS
from bokeh.transform import dodge
from bokeh.io import push_notebook, output_notebook
from datetime import datetime, timedelta
from math import sqrt, log, pi
import localtest
output_notebook()

# Loading and examining the data

Our objective is to study the performance of portfolios drawn from the Dow 30. We retrieved our data from [Quandl](https://www.quandl.com), using the year 2012 for our historical analysis. Feel free to 

In [ ]:
MODEL_START_DATE = '2012-01-01'  # Beginning of historical data pull
MODEL_END_DATE   = '2012-12-31'  # End of historical data pull (inclusive)
FUTURE_END_DATE  = '2013-06-30'  # End of performance test range
QUANDL_KEY = None # Replace with your API string if you want to reload the data from Quandl
SAVE_QUANDL = True # Change to True if you want to save the Quandl query to the cache file

Since the Dow components change periodically, we have accumulated the symbol list for various time periods below. We're choosing our portfolio candidates based on the composition of the Dow on MODEL_END_DATE. Feel free to experiment with arbitrary asset lists (subject to the contents of the Quandl WIKI data set) by replacing this logic with a hardcoded list of strings in the variable `symbol`.

In [ ]:
if MODEL_END_DATE < '2009-06-08':
    raise RuntimeError('No DJIA symbol table for this time frame')
elif MODEL_END_DATE < '2012-09-24':
    # June 8, 2009 - September 24, 2012
    symbols = ['MMM', 'AA', 'AXP', 'T', 'BAC', 'BA', 'CAT', 'CVX', 'CSCO', 'KO',
               'DD', 'XOM', 'GE', 'HPQ', 'HD', 'IBM', 'INTC', 'JNJ', 'JPM', 'KFT',
               'MCD', 'MRK', 'MSFT', 'PFE', 'PG', 'TRV', 'UTX', 'VZ', 'WMT', 'DIS']
elif MODEL_END_DATE < '2013-09-26':
    # September 24, 2012 - September 26, 2013
    symbols = ['MMM', 'AA', 'AXP', 'T', 'BAC', 'BA', 'CAT', 'CVX', 'CSCO', 'KO',
               'DD', 'XOM', 'GE', 'HPQ', 'HD', 'INTC', 'IBM', 'JNJ', 'JPM', 'MCD',
               'MRK', 'MSFT', 'PFE', 'PG', 'TRV', 'UNH', 'UTX', 'VZ', 'WMT', 'DIS']
elif MODEL_END_DATE < '2015-03-19':
    # September 26, 2013 - March 19, 2015
    symbols = ['MMM', 'AXP', 'T', 'BA', 'CAT', 'CVX', 'CSCO', 'KO', 'DD', 'XOM',
               'GE', 'GS', 'HD', 'INTC', 'IBM', 'JNJ', 'JPM', 'MCD', 'MRK', 'MSFT',
               'NKE', 'PFE', 'PG', 'TRV', 'UTX', 'UNH', 'VZ', 'V', 'WMT', 'DIS']
else:
    # March 19, 2015 - present
    symbols = ['MMM', 'AXP', 'AAPL', 'BA', 'CAT', 'CVX', 'CSCO', 'KO', 'DD', 'XOM',
               'GE', 'GS', 'HD', 'INTC', 'IBM', 'JNJ', 'JPM', 'MCD', 'MRK', 'MSFT',
               'NKE', 'PFE', 'PG', 'TRV', 'UNH', 'UTX', 'VZ', 'V', 'WMT', 'DIS']

If `QUANDL_KEY` is supplied, we will go ahead and make a call to the live data feed to retrieve the historical data. Otherise, we will read the data already saved to disk.

In [ ]:
def read_data(filename, start_date, end_date):
    if QUANDL_KEY is not None:
        quandl.ApiConfig.api_key = QUANDL_KEY
        data = quandl.get_table('WIKI/PRICES',
                                qopts = dict(columns=['ticker', 'date', 'adj_close']),
                                ticker = symbols,
                                date = dict(gte=start_date, lte=end_date))
        if SAVE_QUANDL:
            data.to_csv(filename, index=False)
    else:
        data = pd.read_csv(filename, parse_dates=['date'])
    model_start_date = data.date.min().strftime('%Y-%m-%d')
    model_end_date = data.date.max().strftime('%Y-%m-%d')
    return data, model_start_date, model_end_date
raw_data, model_start_date, model_end_date = read_data('input_data.csv', MODEL_START_DATE, MODEL_END_DATE)
raw_data.T

We need to do a bit of data massaging here:

1. Some of the symbols may be missing from the Quandl feed; let's report those.
2. We want each symbol to have its own column; the Pandas `pivot` command makes this nearly automatic. 
3. There are occasional missing values in this data. For now, let us simply fill these gaps with the previous price. Fortunately, the number of such missing elements tend to be small enough that we shouldn't expect too much of an issue.

In [ ]:
def clean_data(data, symbols):
    data = data.pivot(index='date', columns='ticker', values='adj_close')
    missing = ['  - {}'.format(s) for s in symbols if s not in data.columns]
    if missing:
        print('\n'.join(['One or more symbols are missing:'] + missing))
    missing = ['  - {}'.format(s) for s in data.columns[data.isnull().any()]]
    if missing:
        print('\n'.join(['Missing data detected for one or more symbols:'] + missing))
        data.fillna(method='pad', inplace=True)
    return data

data = clean_data(raw_data, symbols)
data.head()

We can look at this data in a variety of ways. First, let us examine some bulk statistics. Inspired by our plan to do mean-variance portfolio analysis, we compute the daily change of each stock, as defined by $c(k) = x(k)/x(k-1) - 1$, and examine the mean and standard deviation of this change over the course of the year. And we also look at the total performance of the stock over the course of the year.

In [ ]:
syms = data.columns
def performance_statistics(data):
    change = (data / data.shift() - 1).iloc[1:]
    stats = pd.concat((change.mean(), change.std(), data.iloc[-1] / data.iloc[0] - 1.0),axis=1)
    stats.columns = ['Mean return', 'Standard deviation', 'Total return']
    return stats
stats = performance_statistics(data)
stats.T

From this, we can summarize the winners and losers.

In [ ]:
def performance_extremes(stats):
    extremes = pd.concat((stats.idxmin(),stats.min(),stats.idxmax(),stats.max()),axis=1)
    extremes.columns = ['Minimizer','Minimum','Maximizer','Maximum']
    return extremes
extremes = performance_extremes(stats)
extremes

Tables are great, but visualizations can often be more more illuminating. Let's plot the percentage growth of each stock over the course of the year. As we can see, the stocks with the best and poorest performance also tend to be the most volatile. We've added hover functionality to most of our plots here: over over a line or data point to see relevant information about the data at that location.

In [ ]:
esyms = list(pd.concat((extremes.Minimizer, extremes.Maximizer)).drop_duplicates())
def performance_multiline(data, esyms, legend=False):
    fig = figure(x_axis_type='datetime')
    growth = data / data.iloc[0] - 1.0
    source = ColumnDataSource(growth)
    line_colors = list(Set1[len(esyms)+1])
    # Lines
    legend_text = None
    for symb in data.columns:
        if symb in esyms:
            alpha, lwid, color = 1, 2, line_colors.pop()
            if legend: legend_text = ' ' + symb
        else:
            alpha, lwid, color, legend_text = 0.25, 1, line_colors[0], None
        r = fig.line('date', symb, color=color, alpha=alpha, line_width=lwid,
                     legend=legend_text, source=source)
        hover = HoverTool(renderers=[r], tooltips=[(symb, '(@date{%F}, @' + symb + ')')],
                          formatters={'date': 'datetime'}, toggleable=False)
        fig.add_tools(hover)
    # Text
    if not legend:
        fig.text(growth.index[-1], growth.iloc[-1],
                 syms, alpha=0.25 + 0.55 * syms.isin(esyms),
                 text_font_size='10px')
    fig.yaxis[0].formatter = NumeralTickFormatter(format="0%")
    if legend:
        fig.legend.orientation='vertical'
        fig.legend.location='top_left'
    return fig
show(performance_multiline(data, esyms))

As we move towards our Markowitz portfolio designs it makes sense to view the means and standard deviations on a scatter plot. If you hover over the individual data points, Bokeh will display a pop-up window with the numeric values and symbol name.

In [ ]:
def performance_scatter(stats, **kwargs):
    fig = figure(**kwargs)
    source = ColumnDataSource(stats)
    hover = HoverTool(tooltips=[('Symbol','@ticker'),
                                ('Standard deviation','@{Standard deviation}'),
                                ('Mean return','@{Mean return}')])
    fig.add_tools(hover)
    fig.circle('Standard deviation', 'Mean return', size=8, color='maroon', source=source, hover_fill_color='grey')
    fig.text('Standard deviation', 'Mean return', 'ticker', text_font_size='10px', x_offset=4, y_offset=-2, source=source)
    fig.xaxis.axis_label='Volatility (standard deviation)'
    fig.yaxis.axis_label='Mean return'
    fig.xaxis[0].formatter = NumeralTickFormatter(format="0.0%")
    fig.yaxis[0].formatter = NumeralTickFormatter(format="0.0%")
    return fig
show(performance_scatter(stats))

As we mentioned in the introduction, a key features of the mean-variance model is the exploitation of _correlations_ between stocks to reduce risk. If two stocks reliably overperform or underperform together (compared to their mean), then they are said to be _positively correlated_. On the other hand, if one stock reliably overperforms when the other underperforms, and vice versa, then they are said to be _negatively correlated_. A mathematical measure of these correlations requires just a single call to Pandas:

In [ ]:
corr = (data / data.shift()).corr()
corr.head()

A visualization has considerable interpretive value here. In the figure below, blue circles represent positive correlations, and red ones represent negative correlations. (For many data sets, there will be few or none of the latter.) The size of the circle represents the strength of the correlation, with the largest possible correlation matching the size of the lighter circles along the diagonal. With just a glance, we can see that stocks like GE and MMM tend to be correlated with the overall performance of the market, while CSCO, UNH, and WMT seem to perform in nearly independent fashion.

In [ ]:
def correlation_scatter(corr):
    nsym = len(corr)
    dsyms = corr.columns.values
    x = np.broadcast_to(dsyms[:, None], (nsym, nsym)).ravel()
    y = np.broadcast_to(dsyms[None, :], (nsym, nsym)).ravel()
    z = corr.values.ravel()
    c = np.where(z < 0, 'red', 'blue')
    a = np.where(x == y, 0.1, 0.8)
    source = ColumnDataSource({'x': x, 'y': y, 'z': z, 'r': z / 2, 'color': c, 'alpha': a})
    fig = figure(x_range=dsyms, y_range=dsyms[::-1])
    fig.add_tools(HoverTool(tooltips='(@y,@x): @z'))
    fig.scatter('x', 'y', radius='r', fill_color='color', alpha='alpha', source=source)
    fig.xaxis.major_label_orientation = pi / 4
    return fig
show(correlation_scatter(corr))

## Building optimized models

Time to bring out the big guns, and talk about what it means to build and _optimal_ model. As the name _mean-variance portfolio optimization_ implies, these models address two objectives: _maximizing performance_, as expressed by the mean return of the portfolio, and _minimizing risk_, as expressed by its variance. Because these objectives are in conflict with each other, portfolio design becomes a study of _tradeoffs_. We can express this tradeoff in two equivalent ways:

- _Risk minimization_. For a given target performance level (mean), minimize the risk (variance).
- _Performance maximization_. For a given target risk tolerance (variance), maximize the performance (mean).

In a practical sense, these two formulations are _equivalent_. For instance, suppose you select a target performance level and solve the risk minimization problem. The value of risk/variance that you obtain can then be fed into the _performance maximization_ model; and subject to very mild conditions, you will get your _original_ performance back! What this means is that you are free to use either type when studying the tradeoffs. In practice, the risk minimization model is often, but not always, the simplest formulation to handle, and we will do so here.

Here is a simple formulation of the risk minimization model:

$$\begin{array}{llll}
\text{minimize}   & x^T \Sigma x \\
\text{subject to} & r^T x = \gamma & & \text{target return} \\
                  & \mathbf{1}^T x = 1 & \quad &\text{fixed budget} \\
                  & x \geq 0       & & \text{no short sales}
\end{array}$$

In this model, the optimization variables $x\in\mathbb{R}^N$ is a vector representing the fraction of our total budget allocated to each stock: that is, $x_i$ is the fraction allocated to stock $i$. The paramters of the model are the mean returns $r\in\mathbb{R}^N$, a *covariance matrix* $\Sigma$, and the target return $\gamma$.

Note that the inclusion of the $x\geq 0$ constraint in the model precludes _short selling_: that is, selling a security that we do not yet own, in hopes of buying it back later a at a lower price. We will examine that assumption in a little more detail later, but relaxing it would complicate our mathematical model for now.

## An initial model:  minimum risk

For our first model, we will actually _drop_ the minimum return constraint, and find the true _minimum risk_ model. That is: if we are forced to purchase a portfolio, and we did not care about the return, what would give us the minimum variance? (If we were not forced to invest, we could achieve zero risk by not doing so.)

So we do not need the mean return vector $r$ just yet---just the covariance matrix $\Sigma$. This matrix is closely related to the cross-correlation matrix computed above, only it is not "normalized". Its diagonal is exactly equal to the variance (the square of the standard deviation) of each stock. It too is computed with a single call to Pandas:

In [ ]:
Sigma = (data / data.shift()).cov()
Sigma.head()

We will be using the CVXOPT method `cvxopt.solvers.qp(P, q, G, h, A, b)` to solve our problem. The optimization model it solves is definitely close to what we want:

$$\begin{array}{lll}
\text{minimize}   & \tfrac{1}{2} x^T P x + q^T x \\
\text{subject to} & G x \leq h \\
                  & A x = b \\
\end{array}$$

What we need to do here is convert our constraints to this form. BWe'll use $G x \leq h$ to implement our no-short constraint $x \geq 0$, and $A x = b$ to implement our budget constraint $\sum_i x_i = 1$. Let us note a couple of minor implementation details:

- Once CVXOPT is finished, we will need to do a little "cleanup" of the solution. At the ideal solution, many of the allocations $x_i$ should be exactly zero; but due to roundoff error, the will have some very small nonzero value instead. So we'll zero those out and re-normalize to get our final portfolio.
- CVXOPT was first released in 2004 _before_ the proliferation of the NumPy array processing library, so it implements its own matrix objects. We can construct our coefficient matrices using NumPy, but we need to convert them to `cvxopt.matrix` objects just prior to calling the solver.

In [ ]:
cvxopt.solvers.options['abstol'] = cvxopt.solvers.options['reltol'] = cvxopt.solvers.options['feastol'] = 1e-8
n = data.shape[1]
P = 2 * cvxopt.matrix(Sigma.values)
q = cvxopt.matrix(np.zeros(n))
G = cvxopt.matrix(-np.eye(n,n))
h = cvxopt.matrix(np.zeros(n))
A = cvxopt.matrix(np.ones((1,n)))
b = cvxopt.matrix(np.ones(1))
solution = cvxopt.solvers.qp(P, q, G, h, A, b)
sol_x = np.array(solution['x'])[:,0]
sol_x *= (sol_x > 1e-4)
min_risk = sol_x / sol_x.sum()

## Adding a performance bound

For the next portfolio, we want a return that is 50% of the mean return obtained by the best-performing stock `BAC`, but with minimum risk. To do this, we need to add an additional equality constraint&mdash; and therefore an additional row to the $A$ and $b$:

In [ ]:
# Add the return target
ret50 = 0.5 * extremes.loc['Mean return', 'Maximum']
r = stats['Mean return'].values
A = cvxopt.matrix(np.vstack([np.ones(n), r]))
b = cvxopt.matrix([1, ret50])
solution = cvxopt.solvers.qp(P, q, G, h, A, b)
sol_x = np.array(solution['x'])[:,0]
sol_x *= (sol_x > 1e-4)
half_ret = sol_x / sol_x.sum()

## Comparing results

Let's compare these two results with the _best performing_ portfolio, which is simply a full allocation devoted to the best-performing stock, BAC, and an _equal allocation_ portfolio, where we invest equally in all stocks.

In [ ]:
best_sym = extremes.loc['Mean return', 'Maximizer']
max_ret = np.array([1.0 if x == best_sym else 0.0 for x in syms])
equal_alloc = np.ones(len(max_ret)) / len(max_ret)

For ease of comparison, we put these into a Pandas dataframe.

In [ ]:
portfolios = pd.DataFrame.from_dict(dict((
    ('Minimum risk', min_risk),
    ('50% Max', half_ret),
    ('Maximum return', max_ret),
    ('Equal allocation', equal_alloc),
    ('ticker', stats.index)))).set_index('ticker')
portfolios.T

A bar chart illustrates the different portfolio allocations. The maximum return portfolio is the only one with a single stock: the 50% risk and minimum risk portfolios have 7 and 11 stocks, respectively.

In [ ]:
def portfolio_bars(portfolios):
    nstocks, nports = portfolios.shape
    width = 0.9 / nports
    left = 0.5 * (1 - width * nports)
    source = ColumnDataSource(portfolios)
    fig = figure(x_range=source.data['ticker'], y_range=(0,1), plot_height=250)
    for k, col in enumerate(portfolios.columns):
        fig.vbar(x=dodge('ticker', left, fig.x_range), top=col, width=width,
                 bottom=0, color=Set1[nports][k], legend=' '+col, source=source)
        left += width
    fig.xaxis.major_label_orientation = pi / 4
    fig.legend.orientation='vertical'
    fig.legend.location='top_right'
    return fig
show(portfolio_bars(portfolios))

Let's look at the same bulk statistics for our porfolios that we considered for the individual stocks. It is important to note, however, that these are quite impractical results, because we are studying the performance of portfolios designed using *the very data fed to the optimization*. In effect, these portfolio designs have the benefit of future knowledge. So this really should be used more as a _sanity check_---that the optimiziation did what we asked---and not as an indicator of future performance. We will look at future performance in a later section.

In [ ]:
port_data = (data / data.iloc[0]).dot(portfolios)
port_stats = performance_statistics(port_data)
port_stats.T

Subject to our caveat above of course, our portfolios live up to their names, and reflect their tradeoffs as well: the minimum risk portfolio has the lowest return, and the maximum return portfolio has the highest risk.

In [ ]:
port_extremes = performance_extremes(port_stats)
port_extremes

Time to plot the results. 

In [ ]:
show(performance_multiline(port_data, port_data.columns, True))

As we can see, the `BAC` singleton portfolio performs the best, as predicted, but it is also quite volatile. The minimum volatility portfolio has the lowest performance of the bunch, even compared to the equal allocation portfolio."

## The efficient frontier

Now we will compute what Modern Portfolio Theory dubs _the efficient frontier_. The efficient frontier is a family of portfolios that represent the _best_ tradeoffs between performance and risk. Any portfolio on the efficient frontier _cannot be improved_ in one criterion (risk or return) without degrading its performance in the other (return or risk). The equal allocation portfolio, for instance, is _not_ on the efficient frontier, because we can either obtain higher performance for no additional risk, or lower risk without sacrificing performance. On the other hand, our minimum risk portfolio and our so-called "50% Max" portfolio are both on the efficient frontier.

To study the entire efficient frontier, we will solve about 100 different optimization models, sweeping the return target from its smallest value to its largest

In [ ]:
# Turn off CVXOPT's output
cvxopt.solvers.options['show_progress'] = False

# Determine the range of returns. Make sure to include the lowest-risk
# portfolio in the list of options
minret = extremes.loc['Mean return','Minimum']
maxret = extremes.loc['Mean return','Maximum']
riskret = extremes.loc['Standard deviation','Minimizer']
riskret = stats.loc[riskret, 'Mean return']
returns = np.unique(np.hstack((np.linspace(minret,maxret,200),riskret)))[::-1]

# Iterate through all returns
risks = returns.copy()
port_data = np.zeros((len(returns), n))
for k in range(len(returns)):
    A = cvxopt.matrix(np.vstack([np.ones(n), r]))
    b = cvxopt.matrix([1, returns[k]])
    solution = cvxopt.solvers.qp(P, q, G, h, A, b)
    sol_x = np.array(solution['x'])[:,0]
    sol_x = np.array(solution['x'])[:,0]
    sol_x *= (sol_x > 1e-4)
    sol_x /= sol_x.sum()
    port_data[k] = sol_x
    risks[k] = np.sqrt(Sigma.dot(sol_x).dot(sol_x))

port_data = pd.DataFrame(port_data.T, columns=[str(x) for x in range(len(returns))], index=syms)

The easiest way to visualize the efficient frontier is to plot it on top of the same mean/volatility scatter plot we created for the individual stocks. When we do this, we see a curve that vaguely justifies why the efficient frontier is often called a "bullet". Each point on the bullet curve represents a particular portfolio that minimizes the risk for a fixed level of return. (To be fair, it is strange to even _consider_ the portfolios on the bottom half of the bullet, but they are well-posed mathematically.)

But we would also like a way to see what the portfolios are at any given point on the tradeoff curve. So we've done a bit of wizardy below, by linking the two plots together below. As you hover over the left-hand plot, the right-hand plot will update with the efficient portfolio that achiaves that particular return. Note that some stocks are _never_ represented in any portfolio on the curve, so they have been omitted from the chart.

In [ ]:
psource = port_data[port_data.any(axis=1)].copy()
psource['_top'] = psource.iloc[:, 0]
psource = ColumnDataSource(psource)
port = figure(x_range=psource.data['ticker'], y_range=(0,1), plot_width=450, plot_height=450)
port_bars = port.vbar(x='ticker', top='_top', width=0.75, bottom=0, source=psource)
port.xaxis.major_label_orientation = pi / 4

# Individual stocks
bullet = figure(plot_width=450, plot_height=450)
bullet.circle('Standard deviation', 'Mean return', size=5, color='maroon', source=stats)
bullet.text('Standard deviation', 'Mean return', 'ticker', text_font_size='10px',
            x_offset=4, y_offset=-2, source=stats)
bullet.xaxis.axis_label='Volatility (standard deviation)'
bullet.yaxis.axis_label='Mean return'
bullet.xaxis[0].formatter = NumeralTickFormatter(format="0.0%")
bullet.yaxis[0].formatter = NumeralTickFormatter(format="0.0%")

# Divide the efficient frontier into two sections: those with
# a return less than the minimum risk portfolio, those that are greater.
tpos_n = returns >= riskret
tneg_n = returns <= riskret
pline = bullet.line(risks[tpos_n], returns[tpos_n], color='blue')
nline = bullet.line(risks[tneg_n], returns[tneg_n], color='red')
bdots = bullet.circle(risks, returns, alpha=0, size=10)
bullet.add_tools(CrosshairTool(dimensions='width'))
bullet.add_tools(HoverTool(mode='hline', renderers=[pline, nline],
                           tooltips={'Volatility': '$x', 'Return': '$y'}))

def callback(source=psource):
    indices = cb_data.index['1d'].indices;
    if indices.length > 0:
        data = source.data;
        data['_top'] = data[indices[0]]
        source.trigger('change')

bullet.add_tools(HoverTool(mode='hline', renderers=[bdots], tooltips=None,
                           callback=CustomJS.from_py_func(callback)))

show(gridplot([[bullet, port]]))

## The beautiful letdown: future performance

Above, we pointed out that our performance plot is unrealistic, because the portfolio designs rely on the very same time period used to built th plot. Let's correct this by examining how the portfolios would have performed in the six months that followed.

In [ ]:
future_start_date = datetime.strptime(model_end_date, '%Y-%m-%d') + timedelta(days=1)
new_data, future_start_date, future_end_date = read_data('future_data.csv', future_start_date, FUTURE_END_DATE)
new_data = clean_data(new_data, symbols)
new_data.head()

In [ ]:
show(performance_multiline(new_data, esyms))

In [ ]:
fport_data = (new_data / new_data.iloc[0]).dot(portfolios)
fport_stats = performance_statistics(fport_data)
fport_stats.T

In [ ]:
fport_extremes = performance_extremes(fport_stats)
fport_extremes

In [ ]:
show(performance_multiline(fport_data, fport_data.columns, True))

Interestingly, the BAC singleton portfolio performs the _worst_ of the bunch. The other remaining portfolios all performed approximately identically---including the equal allocation portfolio. It would seem, alas, that with this particular scenario, our modeling efforts yielded little practical fruit.

Despite our poor showing here, mean variance portfolio optimization remains an important area of study and practice, especially given a variety of improvements and extensions that have been offered since its initial conception. Mathematical optimization remains an absolutely essential component of the portfolio modeling toolchest. And effective visualizations can often provide insights that no amount of tabular data study can reveal. And of course, the combination of all of these advanced computational and visual tools in Jupyter notebooks provide a powerful way to perform experimentation and design, sharable fashion.